## sentiment analysis for fox news

- Vader sentime

In [51]:
#Data Analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

'''Classifiers'''
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import RidgeClassifier , LogisticRegression

'''Metrics/Evaluation'''
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from scipy import interp
from itertools import cycle

'''Plotting'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

'''Display'''
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format

from collections import Counter

### data loading

uskr: 1.0: 47, 0.0: 12(96), -1.0: 17 - 0.65(0.73)
usjp: 1.0: 74, 0.0: 46(50), -1.0: 34 - 0.69(0.68)
jpkr:  0: 88, -1: 78 - 0.92	
cnkr: 0: 116, -1: 10, 1: 9, 0.95 
cnjp: 1: 15, 0: 35, 1: 4 - 0.65
uscn: 

In [21]:
usjp = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_usjp.csv')
uskr = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_uskr.csv')
jpkr = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_jpkr.csv')
cnkr = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_cnkr.csv')
cnjp = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_cnjp.csv')
uscn = pd.read_csv('C:/Users/Juwon Oh/Documents/github/sentiment analysis/sentiment_six_relations_original_uscn.csv')
fox = uskr[['sentiment','text_raw']]
#fox = fox.dropna()
fox = fox.fillna(0)
labels = fox.sentiment.values
text = fox.iloc[:, -1].values
fox.shape

(160, 2)

In [52]:
Counter(labels)

Counter({1.0: 47, 0.0: 96, -1.0: 17})

In [3]:
## stemmer Lemmatizer

In [22]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

new_corpus=[' '.join([stemmer.stem(word) for word in text.split(' ')])for text in text]

In [23]:
from nltk.stem import WordNetLemmatizer
lemmer=WordNetLemmatizer()

new_corpus=[' '.join([lemmer.lemmatize(word) for word in text.split(' ')])for text in new_corpus]

### vectorization and cross validation

In [24]:
count_vect = CountVectorizer(stop_words="english", ngram_range=(1, 2), max_df= 0.95, min_df=4)
text_vector = count_vect.fit_transform(new_corpus)
tfidf_transformer = TfidfTransformer(use_idf=True)
text_Tfidf = tfidf_transformer.fit_transform(text_vector)

In [25]:
text_Tfidf.shape

(160, 3728)

In [26]:
#Dimenionality reduction. Only using the 100 best features er category

lsa = TruncatedSVD(n_components=1000, 
                   n_iter=10, 
                   random_state=3)

X = lsa.fit_transform(text_Tfidf)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, shuffle = True, random_state=0)

## MinMaxScaler

- uskr: 1.0: 47, 0.0: 12(96), -1.0: 17
- cnkr: 0: 116, -1: 10, 1: 9
- cnjp: 1: 15, 0: 35, 1

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1, -1))
scaler.fit_transform(text_Tfidf)
X_train = scaler.fit_transform(text_Tfidf)

In [93]:
from imblearn.over_sampling import SMOTE
# 모델설정
sm = SMOTE(ratio='auto', kind='regular', k_neighbors=5)

# train데이터를 넣어 복제함
X_resampled, y_resampled = sm.fit_sample(X,list(labels))

print('After OverSampling, the shape of train_X: {}'.format(X_resampled.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(X_resampled.shape))

print("After OverSampling, counts of label '-1': {}".format(sum(y_resampled==-1)))
print("After OverSampling, counts of label '1': {}".format(sum(y_resampled==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_resampled==0)))

After OverSampling, the shape of train_X: (288, 160)
After OverSampling, the shape of train_y: (288, 160) 

After OverSampling, counts of label '-1': 96
After OverSampling, counts of label '1': 96
After OverSampling, counts of label '0': 96


## Dimentionality reduction

 - I don't use it once, but I plan to use it later when I receive a large amount of data.

### basic model evaluation 

In [29]:
#Preliminary model evaluation using default parameters

#Creating a dict of the models
model_dict = {'Dummy' : DummyClassifier(random_state=3),
              'Stochastic Gradient Descent' : SGDClassifier(random_state=3, loss='log'),
              'Random Forest': RandomForestClassifier(random_state=3),
              'Decsision Tree': DecisionTreeClassifier(random_state=3),
              'AdaBoost': AdaBoostClassifier(random_state=3),
              'Gaussian Naive Bayes': GaussianNB(),
              'K Nearest Neighbor': KNeighborsClassifier()}
#X_train = X_train.todense()
# X_test = X_test.todense()
#Function to get the scores for each model in a df
def model_score_df(model_dict):   
    model_name, ac_score_list, p_score_list, r_score_list, f1_score_list = [], [], [], [], []
    for k,v in model_dict.items():   
        model_name.append(k)
        v.fit(X_resampled, y_resampled )
        y_pred = v.predict(X_test)
        ac_score_list.append(accuracy_score(y_test, y_pred))
        p_score_list.append(precision_score(y_test, y_pred, average='macro'))
        r_score_list.append(recall_score(y_test, y_pred, average='macro'))
        f1_score_list.append(f1_score(y_test, y_pred, average='macro'))
        model_comparison_df = pd.DataFrame([model_name, ac_score_list, p_score_list, r_score_list, f1_score_list]).T
        model_comparison_df.columns = ['model_name', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score']
        model_comparison_df = model_comparison_df.sort_values(by='f1_score', ascending=False)
    return model_comparison_df

model_score_df(model_dict)

,model_name,accuracy_score,precision_score,recall_score,f1_score
1,Stochastic Gradient Descent,0.67,0.63,0.49,0.52
4,AdaBoost,0.52,0.46,0.47,0.46
5,Gaussian Naive Bayes,0.62,0.38,0.43,0.40
3,Decsision Tree,0.52,0.36,0.38,0.36
2,Random Forest,0.56,0.29,0.32,0.30
6,K Nearest Neighbor,0.25,0.44,0.38,0.28
0,Dummy,0.29,0.33,0.36,0.28


### gridsearch and parameter optimization
 - BernoulliNB
 - SGDClassifier
 - RandomForestClassifier
 - DecisionTreeClassifier
 - extra treeClassifier
 - RidgeClassifier
 - Logisticregression classifier
 - KNeighborsClassifier
 - LinearSVC
 - GradientBoostingClassifier
 

In [75]:
## OneVsRestClassifier+ SVC

classifier = OneVsRestClassifier(SVC(kernel='linear', probability=True,random_state=3))

y_score = classifier.fit(X_resampled, y_resampled).decision_function(X_test)
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.7083333333333334

In [46]:
## BernoulliNB
# BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)

Bernoullinb = BernoulliNB()
params = {'alpha' : [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0,3.0 ,5.0, 7.0, 8.0,10.0]}

Bernoullinb = GridSearchCV(BernoulliNB(), 
                   params,
                   cv = 5,
                   verbose = 3, 
                   n_jobs = -1)
Bernoullinb_best_model = Bernoullinb.fit(X, labels)
print('GridSearchCV best score : {:.2f}%, best_params : {}'.format(Bernoullinb.best_score_*100, Bernoullinb.best_params_))

Fitting 5 folds for each of 13 candidates, totalling 65 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


GridSearchCV best score : 56.88%, best_params : {'alpha': 0.0}


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:    4.9s finished


In [18]:
from sklearn.metrics import classification_report

In [19]:
means = Bernoullinb_best_model.cv_results_['mean_test_score']
stds = Bernoullinb_best_model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, Bernoullinb_best_model.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))
y_true, y_pred = y_test, Bernoullinb_best_model.predict(X_test)
print(classification_report(y_true, y_pred))


0.723 (+/-0.174) for {'alpha': 0.0}
0.723 (+/-0.174) for {'alpha': 0.0001}
0.723 (+/-0.174) for {'alpha': 0.001}
0.723 (+/-0.174) for {'alpha': 0.01}
0.723 (+/-0.174) for {'alpha': 0.1}
0.726 (+/-0.172) for {'alpha': 0.5}
0.725 (+/-0.171) for {'alpha': 1.0}
0.726 (+/-0.163) for {'alpha': 2.0}
0.727 (+/-0.162) for {'alpha': 3.0}
0.728 (+/-0.160) for {'alpha': 5.0}
0.728 (+/-0.160) for {'alpha': 7.0}
0.728 (+/-0.160) for {'alpha': 8.0}
0.731 (+/-0.151) for {'alpha': 10.0}
              precision    recall  f1-score   support

        -1.0       0.42      0.72      0.53        92
         0.0       0.36      0.21      0.27        81
         1.0       0.44      0.10      0.16        41

   micro avg       0.41      0.41      0.41       214
   macro avg       0.41      0.34      0.32       214
weighted avg       0.40      0.41      0.36       214



## roc curve

y_pred = Bernoullinb_best_model.predict(X_test)
conf_mat = confusion_matrix(y_test, y_pred)
ax = sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels= [-1,0,1], yticklabels= [-1,0,1])


n_classes = [-1,0,1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [48]:
y_pred = SGD.predict(X_test)
from sklearn.metrics import accuracy_score
print('Accuracy : {:.3f}'.format(accuracy_score(y_test, y_pred)))
confusion_matrix(y_test, y_pred)

Accuracy : 0.584


array([[68, 17,  7],
       [28, 43, 10],
       [20,  7, 14]], dtype=int64)

SGD = SGDClassifier(random_state=3, loss='log')
SGD.fit(X_train, y_train)

y_pred = Bernoullinb_best_model.predict(X_test) 
conf_mat = confusion_matrix(y_test, y_pred) 
ax = sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels= [-1,0,1], yticklabels= [-1,0,1])
confusion_matrix(y_true, y_pred)

In [91]:
## SGDClassifier
# SGDClassifier(loss=’hinge’, penalty=’l2’, alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate=’optimal’, eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)

loss =  ['hinge','log']
penalty = ['l2','l1']
alpha = [1e-6, 1e-3, 1e-1, 1e0, 0.0001]
max_iter = [5,10,20,30, 50, 60, 70, 100,1000, 10000]
tol = [None, 1e-3]
eta0 = [0.1, 0.001]
random_state = [3]

SGD = SGDClassifier()

params = dict(loss=loss,
              penalty=penalty,
              alpha=alpha,
              max_iter=max_iter,
              tol=tol,
              random_state=random_state)

SGDgridsearch = GridSearchCV(SGDClassifier(),
                          params,
                          cv = 3,
                          verbose = 1, 
                          n_jobs = -1)

sgd_best_model = SGDgridsearch.fit(X, labels)
print('GridSearchCV best score : {:.2f}%, best_params : {}'.format(SGDgridsearch.best_score_*100, SGDgridsearch.best_params_))

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 679 tasks      | elapsed:   33.9s
[Parallel(n_jobs=-1)]: Done 952 tasks      | elapsed:   55.8s


GridSearchCV best score : 77.50%, best_params : {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 70, 'penalty': 'l2', 'random_state': 3, 'tol': None}


[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  1.4min finished


In [92]:
means = sgd_best_model.cv_results_['mean_test_score']
stds = sgd_best_model.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, sgd_best_model.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

0.656 (+/-0.016) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l2', 'random_state': 3, 'tol': None}
0.656 (+/-0.016) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l2', 'random_state': 3, 'tol': 0.001}
0.681 (+/-0.085) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l1', 'random_state': 3, 'tol': None}
0.681 (+/-0.085) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l1', 'random_state': 3, 'tol': 0.001}
0.694 (+/-0.094) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l2', 'random_state': 3, 'tol': None}
0.694 (+/-0.094) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l2', 'random_state': 3, 'tol': 0.001}
0.694 (+/-0.057) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l1', 'random_state': 3, 'tol': None}
0.694 (+/-0.057) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l1', 'random_state': 3, 'tol': 0.001}
0.706 (+/-0.074) for {'alpha': 1e-06, 'l

In [33]:

y_true, y_pred = y_test, sgd_best_model.predict(X_test)
print(classification_report(y_true, y_pred))


0.859 (+/-0.097) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l2', 'random_state': 3, 'tol': None}
0.859 (+/-0.097) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l2', 'random_state': 3, 'tol': 0.001}
0.844 (+/-0.098) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l1', 'random_state': 3, 'tol': None}
0.844 (+/-0.098) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 5, 'penalty': 'l1', 'random_state': 3, 'tol': 0.001}
0.891 (+/-0.129) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l2', 'random_state': 3, 'tol': None}
0.870 (+/-0.085) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l2', 'random_state': 3, 'tol': 0.001}
0.880 (+/-0.130) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l1', 'random_state': 3, 'tol': None}
0.880 (+/-0.130) for {'alpha': 1e-06, 'loss': 'hinge', 'max_iter': 10, 'penalty': 'l1', 'random_state': 3, 'tol': 0.001}
0.859 (+/-0.082) for {'alpha': 1e-06, 'l

In [44]:
estimator =sgd_best_model.best_estimator_
pred=estimator.predict(X_test)
accuracy_score(y_test, pred)

0.6666666666666666

In [53]:
confusion_matrix(y_test, pred)

array([[ 0,  5,  0],
       [ 3, 28,  1],
       [ 0,  7,  4]], dtype=int64)

In [ ]:
## RandomForestClassifier
# randomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)

bootstrap = [True, False]
max_depth = [5,10,20,30, 50, 100, None]
max_features = ['auto', 'sqrt']
min_samples_leaf = [1, 2, 3, 4]
min_samples_split = [2, 5, 10]
n_estimators = [50, 100, 200, 300, 500 ,600, 800]
random_state = [3]

params = dict(bootstrap = bootstrap,
              max_depth = max_depth,
              max_features = max_features,
              min_samples_leaf = min_samples_leaf,
              n_estimators = n_estimators,
              random_state=random_state)

rfgridsearch = GridSearchCV(RandomForestClassifier(),
                          params, 
                          cv=3,
                          verbose=1, 
                          n_jobs=-1)

rf_best_model = rfgridsearch.fit(X_train, y_train)
print('GridSearchCV best score : {:.2f}%, best_params : {}'.format(rfgridsearch.best_score_*100, rfgridsearch.best_params_))

In [ ]:
## DecisionTreeClassifier## extra tree
# DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

params = {'criterion': ["gini", 'entropy'],
          'min_samples_split':[2,3,5,10,12,15,17,20,25,30,35,40],
          'min_samples_leaf':[1,2,3,4,5,7,9,10,13,15,17,20,23,25,27,30]}

DecisionTree_gridsearch = GridSearchCV(DecisionTreeClassifier(), 
                                       params, 
                                       cv=3,
                                       verbose=1, 
                                       n_jobs=-1)

DecisionTree_best_model = DecisionTree_gridsearch.fit(X_train, y_train)
print('DecisionTree GridSearchCV best score : {:.2f}%, best_params : {}'.format(DecisionTree_gridsearch.best_score_*100, DecisionTree_gridsearch.best_params_))

In [179]:
## extra tree

params = {'n_estimators' : [100, 150,180,190, 200],
          'max_depth': [15,20,25,30,35],
          'min_samples_split':[2,5,10]}

et_gridsearch = GridSearchCV(ExtraTreesClassifier(), 
                  params, 
                  cv=3,
                  verbose=1, 
                  n_jobs=-1)
et_best_model = et_gridsearch.fit(X_train, y_train)
print('extra tree GridSearchCV best score : {:.2f}%, best_params : {}'.format(et_gridsearch.best_score_*100, et_gridsearch.best_params_))

Fitting 3 folds for each of 75 candidates, totalling 225 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-1)]: Done 225 out of 225 | elapsed:   56.3s finished


extra tree GridSearchCV best score : 77.04%, best_params : {'max_depth': 35, 'min_samples_split': 2, 'n_estimators': 200}


In [ ]:
## GradientBoostingClassifier
# GradientBoostingClassifier(loss=’deviance’, learning_rate=0.1, n_estimators=100, subsample=1.0, criterion=’friedman_mse’, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort=’auto’, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001)[source]

from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
params = {'n_estimators' : [10, 20, 30, 50, 100, 150, 200],
          'max_depth': [3, 5,10,15,20,25,30],
          'min_samples_split':[2,5,7, 10,12,15]}

gbgridsearch = GridSearchCV(GradientBoostingClassifier(), params,
                     cv=3,
                     verbose=1,
                     n_jobs=-1)

gb_best_model = gbgridsearch.fit(X_train, y_train)
print('extra tree GridSearchCV best score : {:.2f}%, best_params : {}'.format(gbgridsearch.best_score_*100, gbgridsearch.best_params_))

Fitting 3 folds for each of 294 candidates, totalling 882 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  4.5min


In [ ]:
## RidgeClassifier
# class sklearn.linear_model.RidgeClassifier(alpha=1.0, fit_intercept=True, normalize=False, copy_X=True, max_iter=None, tol=0.001, class_weight=None, solver=’auto’, random_state=None)

params = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
RidgeClassifier()
Ridgegridsearch = GridSearchCV(RidgeClassifier(), params,
                     cv=3,
                     verbose=1,
                     n_jobs=-1)

Ridge_model = Ridgegridsearch.fit(X_train, y_train)
print('Ridge GridSearchCV best score : {:.2f}%, best_params : {}'.format(Ridgegridsearch.best_score_*100, Ridgegridsearch.best_params_))


In [ ]:
## LogisticRegression
# LogisticRegression(penalty=’l2’, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver=’warn’, max_iter=100, multi_class=’warn’, verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)


params = {"solver" : ['lbfgs', 'newton-cg'],
          "C":[1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5, 1e6],
          "penalty":["l2"],
          'multi_class': ['multinomial']}


Logisticgridsearch = GridSearchCV(LogisticRegression(), params,
                     cv=5,
                     verbose=1,
                     n_jobs=-1)

Logistic_model = Logisticgridsearch.fit(X_train, y_train)
print('Logistic GridSearchCV best score : {:.2f}%, best_params : {}'.format(Logisticgridsearch.best_score_*100, Logisticgridsearch.best_params_))


In [57]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [51]:
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 1500]}]
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, y_train)

# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test)

Best score for training data: 0.5653923541247485 

Best C: 100 

Best Kernel: rbf 

Best Gamma: 0.1 



In [53]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train).predict(X_test\



SyntaxError: unexpected EOF while parsing (<ipython-input-53-fbaf57c7859f>, line 5)

In [57]:
kernel = 'linear'
dual = False if X.shape[0] > X.shape[1] else True

estimator_names = ['SVC', 'LinearSVC', 'Bagging+SVC', 'Bagging+LinearSVC', 'Bagging+SVC+Multiprocess', 'Bagging+LinearSVC+Multiprocess']
estimators = [SVC(kernel=kernel),
              LinearSVC(dual=dual),
              BaggingClassifier(SVC(kernel=kernel), n_estimators=10, max_samples=0.1, n_jobs=1),
              BaggingClassifier(LinearSVC(dual=dual), n_estimators=10, max_samples=0.1, n_jobs=1),
              BaggingClassifier(SVC(kernel=kernel), n_estimators=10, max_samples=0.1, n_jobs=10),
              BaggingClassifier(LinearSVC(dual=dual), n_estimators=10, max_samples=0.1, n_jobs=10)]
for name, estimator in zip(estimator_names, estimators):
    estimator.fit(X_resampled, y_resampled)
    score = estimator.score(X_test, y_test) # accuracy
    print('{}\t{:.3f}'.format(name, score))


SVC	0.729
LinearSVC	0.729
Bagging+SVC	0.688
Bagging+LinearSVC	0.729
Bagging+SVC+Multiprocess	0.271
Bagging+LinearSVC+Multiprocess	0.729


In [90]:
estimator = BaggingClassifier(SVC(kernel=kernel), n_estimators=100, max_samples=0.5, n_jobs=10)
estimator.fit(X_resampled, y_resampled)
pred=estimator.predict(X_test)
accuracy_score(y_test, pred)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         0.0       0.76      0.91      0.83        32
         1.0       0.88      0.64      0.74        11

   micro avg       0.75      0.75      0.75        48
   macro avg       0.55      0.51      0.52        48
weighted avg       0.71      0.75      0.72        48



In [72]:
confusion_matrix(y_test, pred)

array([[ 0,  5,  0],
       [ 2, 29,  1],
       [ 0,  5,  6]], dtype=int64)

In [51]:
## LinearSVC
# LinearSVC(penalty=’l2’, loss=’squared_hinge’, dual=True, tol=0.0001, C=1.0, multi_class=’ovr’, fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0, random_state=None, max_iter=1000)
labels[labels > 0] = 0



Xs_train, Xs_test, ys_train, ys_test = train_test_split(X, labels, test_size=0.3, shuffle = True, random_state=0)

params: {}



LinearSVCgridsearch = GridSearchCV(LinearSVC(random_state=0, multi_class = 'rammer_singer'), params,
                     cv=3,
                     verbose=1,
                     n_jobs=-1)

LinearSVC_best_model = LinearSVCgridsearch.fit(X_train, y_train)
print('LinearSVC GridSearchCV best score : {:.2f}%, best_params : {}'.format(LinearSVCgridsearch.best_score_*100, LinearSVCgridsearch.best_params_))


Fitting 3 folds for each of 22 candidates, totalling 66 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter solver for estimator LinearSVC(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='multinomial', penalty='l2', random_state=0, tol=0.0001,
     verbose=0). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:

LinearSVC()
    
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))
y_score = classifier.fit(X_train, y_train).decision_function(X_test)
    
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [55]:
## KNeighborsClassifier
# KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)


params = {'n_neighbors' : [2,3,4,5,6,7,9],
          'leaf_size' :[10,20,30,40,50],
          'p': [1,2,3]}

gbgridsearch = GridSearchCV(KNeighborsClassifier(), params,
                     cv=3,
                     verbose=1,
                     n_jobs=-1)

gb_best_model = gbgridsearch.fit(X_train, y_train)
print('KNeighborsClassifier GridSearchCV best score : {:.2f}%, best_params : {}'.format(gbgridsearch.best_score_*100, gbgridsearch.best_params_))


Fitting 3 folds for each of 105 candidates, totalling 315 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.9min


KeyboardInterrupt: 

#### RadiusNeighborsClassifier
- parameter: RadiusNeighborsClassifier(radius=1.0, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, outlier_label=None, metric_params=None, n_jobs=None, **kwargs)

- caution: 


params = {'radius' :[3,2,1.0,0.8,0.5,0.3],
          'weights' : ['uniform', 'distance'],
          'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
          'p': [1,2,3]}

RadiusNeighbors_gridsearch = GridSearchCV(RadiusNeighborsClassifier(), params,
                     cv=3,
                     verbose=1,
                     n_jobs=-1)

RadiusNeighbors_best_model = RadiusNeighbors_gridsearch.fit(X_train, y_train)
print('RadiusNeighborsClassifier GridSearchCV best score : {:.2f}%, best_params : {}'.format(RadiusNeighbors_gridsearch.best_score_*100, RadiusNeighbors_gridsearch.best_params_))

## models accuracy and parameters

- KNeighborsClassifier GridSearchCV best score : 77.69%, best_params : {'leaf_size': 10, 'n_neighbors': 2, 'p': 2}
- Logistic GridSearchCV best score : 77.69%, best_params : {'C': 1000.0, 'multi_class': 'multinomial', 'penalty': 'l2', 'solver': 'newton-cg'}
- Ridge GridSearchCV best score : 78.51%, best_params : {'alpha': 0.02}
- gradient boost GridSearchCV best score : 78.51%, best_params : {'max_depth': 15, 'min_samples_split': 10, 'n_estimators': 30}
- extra tree GridSearchCV best score : 76.86%, best_params : {'max_depth': 15, 'min_samples_split': 5, 'n_estimators': 10}
- SGDClassifier GridSearchCV best score : 75.21%, best_params : {'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 1, 'n_estimators': 500, 'random_state': 3}
- DecisionTree GridSearchCV best score : 72.73%, best_params : {'criterion': 'entropy', 'min_samples_leaf': 2, 'min_samples_split': 5}
- BernoulliNB GridSearchCV best score : 80.17%, best_params : {'alpha': 0.01}


### ensemble model

- https://github.com/davidsbatista/machine-learning-notebooks/blob/master/hyperparameter-across-models.ipynb

 ## Voting Ensemble
 
 - grid search로 최적의 param을 찾은 뒤에, voting으로 모델 개선

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier
clfs = [('BB', BernoulliNB(alpha = 0.01)), ('RF', GradientBoostingClassifier(max_depth = 15, min_samples_split= 10, n_estimators =  30)),('Ridge', RidgeClassifier(alpha=  0.02))]

vote_clf = VotingClassifier(clfs) 

In [ ]:
vote_clf.fit(X_train, y_train)
print('Cross Validation Acc : {:.2f}%'.format(vote_clf.score(X_test, y_test)*100))
y_score = vote_clf.fit(X_train, y_train)
y_pred = vote_clf.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
y_b = label_binarize(labels, classes = [-1, 0, -1])

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=y_b))


In [ ]:

print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,nb_predicted))
print(accuracy_score(y_test, nb_predicted))

In [ ]:


n_classes = y_b.shape[1]


In [ ]:


#Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    
#Compute micro-average ROC curve and ROC area
fpr['micro'], tpr['micro'], _ = roc_curve(y_test_b.ravel(), y_score.ravel())
roc_auc['micro'] = auc(fpr['micro'], tpr['micro'])    

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))
print(accuracy_score(y_test, predicted))
predicted_proba

## Reference

- https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

bigram graph and multi classification

- https://github.com/robsalgado/personal_data_science_projects/blob/master/mulitclass_text_class/Multiclass%20Text%20Classification%20Notebook.ipynb

In [82]:
uskr = pd.read_csv('C:/Users/Juwon Oh/Documents/Dropbox/BigDataDiplomacy/Code/2019/Analysis/Fall/USKR.csv', encoding = "ISO-8859-1")

In [ ]:
estimator = BaggingClassifier(SVC(kernel=kernel), n_estimators=100, max_samples=0.5, n_jobs=10)
estimator.fit(X_resampled, y_resampled)
pred=estimator.predict(X_test)
accuracy_score(y_test, pred)

In [88]:
uskr = uskr[['date', 'title', 'text']]
uskr_text= uskr[['text']]

In [89]:
pred=estimator.predict(uskr_text)

ValueError: Number of features of the model must match the input. Model n_features is 160 and input n_features is 1.